# Telco Customer Churn - Exploratory Data Analysis

**Objective:** Understand the customer churn dataset, identify patterns, and prepare for modeling.

**Data Source:** `data/raw/customer_churn_dataset.xlsx`

**Mindset:** Trust nothing, verify everything. Every analysis should answer: *"How does this help me build a better model?"*


## Setup


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# Configure visualization
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
%matplotlib inline

# Suppress warnings
import warnings
warnings.filterwarnings('ignore')


## 1. Load Data


In [ ]:
# Load the Excel file from raw data
data_path = '../../data/raw/customer_churn_dataset.xlsx'

print(f"Loading data from: {data_path}")
df = pd.read_excel(data_path)

print(f"\n✓ Dataset loaded successfully!")
print(f"Shape: {df.shape}")
print(f"Rows: {df.shape[0]:,}")
print(f"Columns: {df.shape[1]}")

# Display first few rows
print("\n=== First 5 Rows ===")
display(df.head())

# Display last few rows
print("\n=== Last 5 Rows ===")
display(df.tail())


## 2. Data Overview


In [ ]:
# Basic info
print("=== Data Types ===")
df.info()

print("\n=== Column Names ===")
print(df.columns.tolist())


In [ ]:
# Statistical summary
df.describe()


In [ ]:
# Categorical summary
print("=== Categorical Features Summary ===")
df.describe(include='object')


### 2.1 Check for Duplicates


In [ ]:
# Check for duplicate rows
duplicates = df.duplicated().sum()
print(f"Duplicate rows: {duplicates}")

if duplicates > 0:
    print(f"⚠️ Warning: {duplicates} duplicate rows found ({duplicates/len(df)*100:.2f}%)")
    print("\nSample duplicates:")
    display(df[df.duplicated(keep=False)].head(10))
else:
    print("✓ No duplicate rows found")


## 3. Data Quality - Missing Values


In [ ]:
# Missing values
missing = pd.DataFrame({
    'Missing Count': df.isnull().sum(),
    'Percentage': (df.isnull().sum() / len(df) * 100).round(2)
})
missing = missing[missing['Missing Count'] > 0].sort_values('Missing Count', ascending=False)

if len(missing) > 0:
    print("⚠️ Missing Values Found:")
    print(missing)
    
    # Visualize
    plt.figure(figsize=(10, 6))
    missing['Percentage'].plot(kind='barh')
    plt.xlabel('Percentage Missing (%)')
    plt.title('Missing Values by Feature')
    plt.tight_layout()
    plt.show()
else:
    print("✓ No missing values found")


## 4. Target Variable Analysis


In [ ]:
# Identify target column (typically 'Churn', 'Churn Label', etc.)
churn_cols = [col for col in df.columns if 'churn' in col.lower()]
print(f"Potential churn columns: {churn_cols}")

# Set target column - adjust based on your data
target_col = churn_cols[0] if churn_cols else 'Churn'

if target_col in df.columns:
    print(f"\n=== Target Variable: {target_col} ===")
    print(df[target_col].value_counts())
    print(f"\nChurn Rate: {(df[target_col].value_counts(normalize=True) * 100).round(2)}")
    
    # Visualize
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Count plot
    df[target_col].value_counts().plot(kind='bar', ax=axes[0], color=['#2ecc71', '#e74c3c'])
    axes[0].set_title('Churn Distribution (Count)', fontsize=14, fontweight='bold')
    axes[0].set_xlabel(target_col)
    axes[0].set_ylabel('Count')
    axes[0].tick_params(axis='x', rotation=0)
    
    # Percentage plot
    df[target_col].value_counts(normalize=True).plot(kind='bar', ax=axes[1], color=['#2ecc71', '#e74c3c'])
    axes[1].set_title('Churn Distribution (Percentage)', fontsize=14, fontweight='bold')
    axes[1].set_xlabel(target_col)
    axes[1].set_ylabel('Proportion')
    axes[1].tick_params(axis='x', rotation=0)
    
    plt.tight_layout()
    plt.show()
else:
    print(f"⚠️ Warning: '{target_col}' column not found. Please identify the target variable.")


### 4.1 Class Balance Check


In [ ]:
# Check class balance
if target_col in df.columns:
    value_counts = df[target_col].value_counts()
    percentages = df[target_col].value_counts(normalize=True) * 100
    
    print("=== Class Distribution ===")
    for val, count in value_counts.items():
        pct = percentages[val]
        print(f"{val}: {count:,} ({pct:.2f}%)")
    
    # Calculate imbalance ratio
    imbalance_ratio = value_counts.max() / value_counts.min()
    print(f"\nImbalance Ratio: {imbalance_ratio:.2f}:1")
    
    if imbalance_ratio > 3:
        print("⚠️ Significant class imbalance detected!")
        print("Consider: SMOTE, class weights, or stratified sampling")
    else:
        print("✓ Classes are reasonably balanced")


## 5. Numerical Features Distribution


In [ ]:
# Get numerical columns
numeric_cols = df.select_dtypes(include=['int64', 'float64']).columns.tolist()
print(f"Numerical columns ({len(numeric_cols)}): {numeric_cols}")

# Plot distributions
if len(numeric_cols) > 0:
    n_cols = 3
    n_rows = (len(numeric_cols) + n_cols - 1) // n_cols
    
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, 5*n_rows))
    axes = axes.flatten() if n_rows > 1 else [axes] if n_cols == 1 else axes
    
    for idx, col in enumerate(numeric_cols):
        axes[idx].hist(df[col].dropna(), bins=30, edgecolor='black', alpha=0.7)
        axes[idx].set_title(f'{col}', fontweight='bold')
        axes[idx].set_xlabel(col)
        axes[idx].set_ylabel('Frequency')
        axes[idx].grid(alpha=0.3)
    
    # Hide unused subplots
    for idx in range(len(numeric_cols), len(axes)):
        axes[idx].set_visible(False)
    
    plt.tight_layout()
    plt.show()


### 5.1 Statistical Summary


In [ ]:
# Detailed statistics for numerical features
if len(numeric_cols) > 0:
    print("=== Numerical Features Statistics ===\n")
    
    stats_df = df[numeric_cols].agg(['count', 'mean', 'median', 'std', 'min', 'max', 'skew']).T
    stats_df = stats_df.round(2)
    display(stats_df)
    
    # Identify skewed features
    skewed_features = stats_df[abs(stats_df['skew']) > 1].index.tolist()
    if skewed_features:
        print(f"\n⚠️ Highly skewed features (|skew| > 1): {skewed_features}")
        print("Consider: Log transformation, Box-Cox, or robust scaling")


## 6. Categorical Features Analysis


In [ ]:
# Get categorical columns
categorical_cols = df.select_dtypes(include=['object']).columns.tolist()

# Remove ID columns if any
categorical_cols = [col for col in categorical_cols if 'id' not in col.lower()]

print(f"Categorical columns ({len(categorical_cols)}): {categorical_cols}")

# Show unique values for each categorical column
for col in categorical_cols[:10]:  # Limit to first 10
    unique_count = df[col].nunique()
    print(f"\n{col}: {unique_count} unique values")
    print(df[col].value_counts().head())


## 7. Correlation Analysis


### 6.1 Cardinality Check


In [ ]:
# Check cardinality for categorical features
if len(categorical_cols) > 0:
    print("=== Categorical Features Cardinality ===\n")
    
    cardinality_df = pd.DataFrame({
        'Feature': categorical_cols,
        'Unique Values': [df[col].nunique() for col in categorical_cols],
        'Sample Values': [', '.join(map(str, df[col].unique()[:3])) + '...' for col in categorical_cols]
    }).sort_values('Unique Values', ascending=False)
    
    display(cardinality_df)
    
    # Flag high cardinality features
    high_card = cardinality_df[cardinality_df['Unique Values'] > 50]['Feature'].tolist()
    if high_card:
        print(f"\n⚠️ High cardinality features (>50 unique): {high_card}")
        print("Consider: Target encoding, frequency encoding, or grouping rare categories")


In [ ]:
# Correlation matrix for numerical features
if len(numeric_cols) > 1:
    plt.figure(figsize=(12, 10))
    correlation_matrix = df[numeric_cols].corr()
    
    sns.heatmap(correlation_matrix, 
                annot=True, 
                fmt='.2f', 
                cmap='coolwarm', 
                center=0, 
                square=True,
                linewidths=0.5)
    plt.title('Feature Correlation Matrix', fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.show()
    
    # Find highly correlated features (>0.8)
    high_corr = []
    for i in range(len(correlation_matrix.columns)):
        for j in range(i+1, len(correlation_matrix.columns)):
            if abs(correlation_matrix.iloc[i, j]) > 0.8:
                high_corr.append((correlation_matrix.columns[i], 
                                correlation_matrix.columns[j], 
                                correlation_matrix.iloc[i, j]))
    
    if high_corr:
        print("\n⚠️ Highly Correlated Features (|r| > 0.8):")
        for feat1, feat2, corr in high_corr:
            print(f"  {feat1} ↔ {feat2}: {corr:.3f}")
    else:
        print("\n✓ No highly correlated features found")


## 8. Churn vs Numerical Features


### 7.1 Top Correlations with Target


In [ ]:
# If target is numeric, show correlations with target
if target_col in df.columns and len(numeric_cols) > 1:
    # Try to encode target if it's categorical
    target_encoded = df[target_col].copy()
    if df[target_col].dtype == 'object':
        # Simple encoding: Yes/No -> 1/0, or use factorize
        if set(df[target_col].unique()).issubset({'Yes', 'No'}):
            target_encoded = (df[target_col] == 'Yes').astype(int)
        else:
            target_encoded = pd.factorize(df[target_col])[0]
    
    # Calculate correlations
    correlations_with_target = df[numeric_cols].corrwith(pd.Series(target_encoded, index=df.index))
    correlations_with_target = correlations_with_target.abs().sort_values(ascending=False)
    
    print("=== Features Most Correlated with Target ===\n")
    print(correlations_with_target.head(10))
    
    # Visualize
    plt.figure(figsize=(10, 6))
    correlations_with_target.head(15).plot(kind='barh', color='skyblue')
    plt.xlabel('Absolute Correlation')
    plt.title('Top 15 Features by Correlation with Target', fontweight='bold')
    plt.gca().invert_yaxis()
    plt.tight_layout()
    plt.show()


In [ ]:
# Box plots: numerical features vs churn
if target_col in df.columns and len(numeric_cols) > 0:
    n_features = min(6, len(numeric_cols))  # Show top 6
    
    fig, axes = plt.subplots(2, 3, figsize=(16, 10))
    axes = axes.flatten()
    
    for idx, col in enumerate(numeric_cols[:n_features]):
        df.boxplot(column=col, by=target_col, ax=axes[idx])
        axes[idx].set_title(f'{col} vs {target_col}')
        axes[idx].set_xlabel(target_col)
        axes[idx].set_ylabel(col)
        plt.sca(axes[idx])
        plt.xticks(rotation=0)
    
    plt.suptitle('Numerical Features vs Churn', fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.show()


## 9. Churn vs Categorical Features


In [ ]:
# Churn rate by categorical features
if target_col in df.columns and len(categorical_cols) > 0:
    n_features = min(6, len(categorical_cols))
    
    fig, axes = plt.subplots(2, 3, figsize=(16, 10))
    axes = axes.flatten()
    
    for idx, col in enumerate(categorical_cols[:n_features]):
        if df[col].nunique() <= 10:  # Only plot if not too many categories
            churn_rate = df.groupby(col)[target_col].apply(
                lambda x: (x == 'Yes').sum() / len(x) if 'Yes' in x.values else x.mean()
            ).sort_values(ascending=False)
            
            churn_rate.plot(kind='bar', ax=axes[idx], color='coral')
            axes[idx].set_title(f'Churn Rate by {col}')
            axes[idx].set_xlabel(col)
            axes[idx].set_ylabel('Churn Rate')
            axes[idx].tick_params(axis='x', rotation=45)
            axes[idx].grid(alpha=0.3, axis='y')
    
    plt.suptitle('Churn Rate by Categorical Features', fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.show()


### 8.1 Statistical Comparison


In [ ]:
# Compare means by target class
if target_col in df.columns and len(numeric_cols) > 0:
    print("=== Mean Values by Target Class ===\n")
    
    comparison = df.groupby(target_col)[numeric_cols].mean().T
    comparison['Difference'] = comparison.iloc[:, 1] - comparison.iloc[:, 0]
    comparison['Pct_Diff'] = (comparison['Difference'] / comparison.iloc[:, 0] * 100).round(2)
    comparison = comparison.reindex(comparison['Pct_Diff'].abs().sort_values(ascending=False).index)
    
    display(comparison.head(10))
    
    print("\n💡 Features with largest mean differences may be good predictors")


### 10.1 Outlier Statistics


In [ ]:
# Calculate outliers using IQR method
if len(numeric_cols) > 0:
    print("=== Outlier Count (IQR Method) ===\n")
    
    outlier_summary = []
    for col in numeric_cols:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        
        outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)]
        outlier_count = len(outliers)
        outlier_pct = (outlier_count / len(df)) * 100
        
        outlier_summary.append({
            'Feature': col,
            'Outlier Count': outlier_count,
            'Percentage': f"{outlier_pct:.2f}%",
            'Lower Bound': f"{lower_bound:.2f}",
            'Upper Bound': f"{upper_bound:.2f}"
        })
    
    outlier_df = pd.DataFrame(outlier_summary).sort_values('Outlier Count', ascending=False)
    display(outlier_df)
    
    high_outliers = outlier_df[outlier_df['Outlier Count'] > len(df) * 0.05]['Feature'].tolist()
    if high_outliers:
        print(f"\n⚠️ Features with >5% outliers: {high_outliers}")
        print("Consider: Capping (winsorization), log transformation, or investigation")


## 10. Outlier Detection


In [ ]:
# Box plots for outlier detection
if len(numeric_cols) > 0:
    n_features = min(6, len(numeric_cols))
    
    fig, axes = plt.subplots(2, 3, figsize=(16, 10))
    axes = axes.flatten()
    
    for idx, col in enumerate(numeric_cols[:n_features]):
        axes[idx].boxplot(df[col].dropna())
        axes[idx].set_title(f'{col}')
        axes[idx].set_ylabel(col)
        axes[idx].grid(alpha=0.3)
    
    plt.suptitle('Outlier Detection via Box Plots', fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.show()


## 11. Key Findings Summary


### 9.1 Statistical Significance


In [ ]:
# Chi-square test for categorical features
from scipy.stats import chi2_contingency

if target_col in df.columns and len(categorical_cols) > 0:
    print("=== Chi-Square Test: Categorical Features vs Target ===\n")
    
    chi_square_results = []
    for col in categorical_cols[:10]:  # Limit to first 10
        if df[col].nunique() <= 20:  # Only test if reasonable number of categories
            contingency_table = pd.crosstab(df[col], df[target_col])
            chi2, p_value, dof, expected = chi2_contingency(contingency_table)
            
            chi_square_results.append({
                'Feature': col,
                'Chi2': chi2,
                'P-Value': p_value,
                'Significant': '✓' if p_value < 0.05 else '✗'
            })
    
    if chi_square_results:
        chi_df = pd.DataFrame(chi_square_results).sort_values('P-Value')
        display(chi_df)
        
        significant_features = chi_df[chi_df['P-Value'] < 0.05]['Feature'].tolist()
        print(f"\n💡 Statistically significant features (p < 0.05): {len(significant_features)}")
        print(f"   {significant_features}")


### 📊 Summary & Action Plan

Fill in after running all cells above:

**✅ Data Quality:**
- Total rows: ___
- Missing values: ___
- Duplicates: ___

**🎯 Target Variable:**
- Distribution: ___
- Imbalance ratio: ___:1
- Baseline accuracy: ___%

**📊 Top 5 Predictive Features:**
1. ___
2. ___
3. ___
4. ___
5. ___

**🔧 Preprocessing Plan:**
- Missing values: ___
- Outliers: ___
- Encoding: ___
- Scaling: ___
- Class imbalance: ___

**🚫 Features to Drop:**
- ___ (reason: ___)

**🚀 Next Steps:**
1. [ ] Run data pipeline: `python src/pipeline.py`
2. [ ] Train baseline model
3. [ ] Track in MLflow

**💡 Key Insights:**
- ___
- ___
- ___

**Data Quality:**
- [ ] Missing values: [describe]
- [ ] Duplicates: [check]
- [ ] Data types: [correct/needs fixing]

**Target Variable:**
- [ ] Class imbalance: [percentage]
- [ ] Action needed: [balancing strategy if needed]

**Features:**
- [ ] Most correlated with churn: [list top 3-5]
- [ ] Highly correlated features: [list pairs to consider removing]
- [ ] Outliers: [significant outliers found in which features?]

**Next Steps:**
1. [ ] Handle missing values (strategy: ___)
2. [ ] Feature engineering ideas: [list]
3. [ ] Features to drop: [list with reasons]
4. [ ] Encoding strategy: [one-hot, label, etc.]
5. [ ] Scaling needed: [yes/no, which features]
